# Script for Preparing NetCDF Data for SNOWPACK - Multiyear CESM2LE
**Description:** Converts NetCDF data to SMET type and imposes user-defined precip cutoff    
**Input Data:** TSA_R, RH2M_R, UBOT, VBOT (all 3-hourly), FSDS, FLDS, TS (all daily), PRECT (hourly)  
**Output Data:** SMET file for SNOWPACK  
**Author:** Emma Perkins  
**Date:** June 2023  

In [1]:
# loading in relevant packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import math
import re
import calendar
import cftime
import glob
#!pip install pysolar
from pysolar.solar import *
import time
import os
import warnings

In [2]:
# parameters for dask optimization
nworkers = 30
chunk_time = 200
chunk_lat = 100
chunk_lon = 100
nmem='20'

In [3]:
# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory=nmem+'GiB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/scratch/$USER/local_dask', # Use your local directory
    resource_spec='select=1:ncpus=1:mem='+nmem+'GB', # Specify resources
    project='P93300065', # Input your project ID here, previously this was known as 'project', now is 'account'
    walltime='04:00:00', # Amount of wall time
    #interface="ib0" # Interface to use
)

# Scale up
cluster.scale(nworkers)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/work/eperkins/miniconda3/envs/analysis3/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45667 instead
  f"Port {expected} is already in use.\n"


In [4]:
client  #wait a cuople mins before running this cell to give workers some time to spin up

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/eperkins/proxy/45667/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/eperkins/proxy/45667/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.100:42547,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/eperkins/proxy/45667/status,Total threads: 0
Started: Just now,Total memory: 0 B


## **Supporting Functions**

In [5]:
def read_data(ensemble_member):
    #function to read in data for all SNOWPACK required variables for a given ensemble member
    LE_pairs = [['01','01'], ['02','03'],['03','05'],['04','07'],['05','09'],['06','11'],['07','13'],['08','15'],['09','17'],['10','19']]
    first = LE_pairs[ensemble_member-1][1]
    second = LE_pairs[ensemble_member-1][0]

    #master paths to directories for different types of variables
    lnd_path_3 = '/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/hour_3/'
    lnd_path_day = '/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/day_1/'
    atm_path_3 = '/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/hour_3/'
    atm_path_day = '/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/'

    #air temp:
    at_names = 'TSA_R/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.clm2.h7.TSA_R.*.nc' #rural 2 meter air temp (3-hourly)
    at_files = sorted(glob.glob(lnd_path_3 + at_names))
    at_data = xr.open_mfdataset(at_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #relative humidity:
    rh_names = 'RH2M_R/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.clm2.h7.RH2M_R.*.nc' #rural 2 meter air temp (3-hourly)
    rh_files = sorted(glob.glob(lnd_path_3 + rh_names))
    rh_data = xr.open_mfdataset(rh_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #u wind:
    u_names = 'UBOT/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.cam.h3.UBOT.*.nc' #bottom level u wind (3- hourly)
    u_files = sorted(glob.glob(atm_path_3 + u_names))
    u_data = xr.open_mfdataset(u_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #v wind:
    v_names = 'VBOT/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.cam.h3.VBOT.*.nc' #bottom level v wind (3-hourly)
    v_files = sorted(glob.glob(atm_path_3 + v_names))
    v_data = xr.open_mfdataset(v_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #incoming shortwave radiation:
    sw_names = 'FSDS/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.cam.h1.FSDS.*.nc' #incoming solar radiation at surface (daily)
    sw_files = sorted(glob.glob(atm_path_day + sw_names))
    sw_data = xr.open_mfdataset(sw_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #incoming longwave radiation:
    lw_names = 'FLDS/b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.cam.h1.FLDS.*.nc' #incoming solar radiation at surface (daily)
    lw_files = sorted(glob.glob(atm_path_day + lw_names))
    lw_data = xr.open_mfdataset(lw_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #total precipitation
    prcp_path = '/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/hour_1/PRECT/'
    prcp_names = 'b.e21.BSSP370smbb.f09_g17.LE2-1'+first+'1.0'+second+'.cam.h4.PRECT.*.nc' #hourly
    prcp_files = sorted(glob.glob(prcp_path + prcp_names))
    prcp_data = xr.open_mfdataset(prcp_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #ground temp
    gt_names = 'TG/b.e21.BSSP370smbb.f09_g17.LE2-1031.002.clm2.h5.TG.*.nc'
    gt_files = sorted(glob.glob(lnd_path_day + gt_names))
    gt_data = xr.open_mfdataset(gt_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})

    #surface temp
    st_names = 'TSKIN/b.e21.BSSP370smbb.f09_g17.LE2-1031.002.clm2.h5.TSKIN.*.nc'
    st_files = sorted(glob.glob(lnd_path_day + st_names))
    st_data = xr.open_mfdataset(st_files, parallel = True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})
    
    return (at_data, rh_data, u_data, v_data, sw_data, lw_data, gt_data, st_data, prcp_data)

In [6]:
def data_prep(at_data, rh_data, u_data, v_data, sw_data, lw_data, gt_data, st_data, prcp_data, lat_coord, lon_coord, precip_cutoff, start, end):
    #function for preparing data read in with read_data() to SNOWPACK specifications
    
    #Unit/Data Type notes:
    #lat_coord: float
    #lon_coord: float
    #precip_cutoff: foat (in mm)
    #start: string of start date in format 'YYYY-MM-DD'
    #end: string of end date in format 'YYYY-MM-DD'
    
    #1. Select only variables of interest - reduces computational cost--------------------------------------------------------------------------------------------------------------
    all_data = [at_data.TSA_R, rh_data.RH2M_R, u_data.UBOT, v_data.VBOT, sw_data.FSDS, lw_data.FLDS, prcp_data.PRECT, gt_data.TG, st_data.TSKIN] #make list of all dataframes and select only variables of interest
    #all_data = [at_data, rh_data, u_data, v_data, sw_data, lw_data, prcp_data, gt_data]
    
    #2. Select only data for location and timeframe of interest and interpolate to hourly resolution--------------------------------------------------------------------------------
    loc_data = list(range(len(all_data))) #initialize list of point-specific data
    for i in range(len(all_data)):
        loc_data[i] = all_data[i].sel(lat = lat_coord, method = 'nearest').sel(lon = lon_coord, method = 'nearest').resample(time='1H').interpolate('cubic').sel(time=slice(start, end))
        loc_data[i]['lat'] = round(float(loc_data[i].lat.values),2) # round all latitudes to 2 decimal places so that they match exactly for data merging
    
    #3. Merge all remaining data into one array and rename variables to match SNOWPACK inputs---------------------------------------------------------------------------------------
    full_data = xr.merge(loc_data)
    full_data = full_data.rename({'TSA_R': 'TA', 'RH2M_R': 'RH', 'TG': 'TSG', 'TSKIN': 'TSS', 'FSDS' : 'ISWR', 'FLDS': 'ILWR', 'UBOT': 'U', 'VBOT': 'V'})
    #print('Done with point selection')

    #4. Calculate clear sky ISWR-----------------------------------------------------------------------------------------------------------------------------------------------------
    solar_alt = get_altitude_fast(lat_coord, lon_coord, full_data.indexes['time'].to_datetimeindex().tz_localize('UTC')) #create numpy of solar altitude for each time step
    data_solar = xr.DataArray(solar_alt, coords = {'lon': full_data.lon.values, 'lat': full_data.lat.values, 'time':  full_data.indexes['time']}, 
                              dims = ['time'], name = 'solar_alt') #convert numpy array to xarray for merging with full_data
    full_data = xr.merge([full_data, data_solar])
    full_data = full_data.assign(ISWR_clear = lambda df: radiation.get_radiation_direct(df.time, df.solar_alt)) #calculate clear sky radiation from solar altitude
    full_data['ISWR_clear'] = full_data.ISWR_clear.where(full_data.solar_alt >= 0, other = 0) #articficially make ISWR 0 when solar altitude is below horizon since pysolar will not automatically do this

    #5. correct clear sky ISWR according to CESM2 daily mean------------------------------------------------------------------------------------------------------------------------
    #create arrays with the true and clear mean of each day repeated 24 times (once per hour) to concatenate with hourly xarray dataframe
    ISWR_true_mean_repeat = np.repeat(sw_data.FSDS.sel(lat = lat_coord, method = 'nearest').sel(lon = lon_coord, method = 'nearest').sel(time=slice(start, end)).values, 24)
    ISWR_clear_mean_repeat = np.repeat(full_data.ISWR_clear.resample(time='1D').mean(dim='time').values, 24)

    #turn mean numpy arrays into xarray format
    ISWR_data_clear = xr.DataArray(ISWR_clear_mean_repeat, coords = {'lon': full_data.lon.values, 'lat': full_data.lat.values, 'time':  full_data.indexes['time']}, 
                              dims = ['time'], name = 'ISWR_clear_mean')
    ISWR_data_true = xr.DataArray(ISWR_true_mean_repeat, coords = {'lon': full_data.lon.values, 'lat': full_data.lat.values, 'time':  full_data.indexes['time']}, 
                              dims = ['time'], name = 'ISWR_true_mean')

    #merge means with rest of the data
    full_data = xr.merge([full_data, ISWR_data_clear, ISWR_data_true])

    #calculate corrected ISWR using ISWR_clear and both means
    full_data = full_data.assign(ISWR_corrected = lambda df: df.ISWR_clear - df. ISWR_clear) #create variable of all zeros
    full_data['ISWR_corrected'] = full_data.ISWR_corrected.where(full_data.ISWR_clear_mean == 0, other = full_data.ISWR_clear*full_data.ISWR_true_mean/full_data.ISWR_clear_mean)  
        #in places where clear mean isn't zero, adjust ISWR_clear by true_mean/clear_mean
        #leave ISWR_corrected as 0 in places where clear_mean is zero because that means there is no incident radiation and dividing by zero gives nans
    
    full_data = full_data.drop('ISWR')
    full_data = full_data.rename({'ISWR_corrected': 'ISWR'})
    #print('Done with ISWR correction')
    
    
    #6. Calculate clear sky ILWR according to Idso and Jackson (1969)
    full_data = full_data.assign(ILWR_clear = lambda df: 5.67*10**(-8)*(df.TA**4)*(1-0.261*np.exp(-7.77*10**(-4)*(273-df.TA)**2)))

    #7. Correct clear sky ILWR according to CESM2 daily mean
    #create arrays with the true and clear mean of each day repeated 24 times (once per hour) to concatenate with hourly xarray dataframe
    ILWR_true_mean_repeat = np.repeat(lw_data.FLDS.sel(lat = lat_coord, method = 'nearest').sel(lon = lon_coord, method = 'nearest').sel(time=slice(start, end)).values, 24)
    ILWR_clear_mean_repeat = np.repeat(full_data.ILWR_clear.resample(time='1D').mean(dim='time').values, 24)

    #turn mean numpy arrays into xarray format
    ILWR_data_clear = xr.DataArray(ILWR_clear_mean_repeat, coords = {'lon': full_data.lon.values, 'lat': full_data.lat.values, 'time':  full_data.indexes['time']}, 
                              dims = ['time'], name = 'ILWR_clear_mean')
    ILWR_data_true = xr.DataArray(ILWR_true_mean_repeat, coords = {'lon': full_data.lon.values, 'lat': full_data.lat.values, 'time':  full_data.indexes['time']}, 
                              dims = ['time'], name = 'ILWR_true_mean')

    #merge means with rest of the data
    full_data = xr.merge([full_data, ILWR_data_clear, ILWR_data_true])

    #calculate corrected ISWR using ISWR_clear and both means
    full_data = full_data.assign(ILWR_corrected = lambda df: df.ILWR_clear - df. ILWR_clear) #create variable of all zeros
    full_data['ILWR_corrected'] = full_data.ILWR_corrected.where(full_data.ILWR_clear_mean == 0, other = full_data.ILWR_clear*full_data.ILWR_true_mean/full_data.ILWR_clear_mean)
        #in places where clear mean isn't zero, adjust ISLR_clear by true_mean/clear_mean
        #leave ILWR_corrected as 0 in places where clear_mean is zero because that means there is no incident radiation and dividing by zero gives nans
        
    full_data = full_data.drop('ILWR')
    full_data = full_data.rename({'ILWR_corrected': 'ILWR'})
    #print('Done with ILWR correction')
    
    #6. Convert wind components to total velocity (VW) and direction (DW)-----------------------------------------------------------------------------------------------------------
    full_data['VW'] = ((full_data.U)**2 + (full_data.V)**2)**(1/2)
    full_data['DW'] = np.arctan(full_data.V / full_data.U)
    #print('Done with wind components conversion')

    #7. convert precipitation and impose cutoff
    full_data['PSUM'] = full_data.PRECT * 3.6 * 10**6 #convert m/s to cummulative precip (mm)
    full_data['PSUM'] = full_data.PSUM.where(full_data.PSUM >= precip_cutoff, other = 0)
    #print('Done with precip cutoff')

    #8. convert RH from percent to decimal
    full_data['RH'] = full_data.RH / 100

    #9. drop all intermediate variables
    data4analysis = full_data[['TA', 'RH', 'TSG', 'TSS', 'PSUM', 'VW', 'DW', 'ILWR', 'ISWR']]
    #data4analysis = full_data.drop(['true_mean', 'clear_mean', 'solar_alt', 'VBOT', 'UBOT', 'ISWR_clear', 'PRECT'])    
    
    #10. account for leap years
    times = data4analysis.indexes['time'].to_datetimeindex()
    new = np.empty(times.shape, dtype="O")

    for i, t in enumerate(times):
        dt = cftime.DatetimeAllLeap(t.year, t.month, t.day, t.hour, t.minute, t.second, t.microsecond) #convert calendar type to 'all_leap' from 'no_leap'
        new[i] = dt
        
    data4analysis['time'] = new #assign new calendar type (still has exact same dates)
    
    datetimeleap = xr.cftime_range(start = start, end = end[:-1]+str(int(end[-1])+1), 
                    calendar = 'all_leap', freq='H') #create new calendar where every year has a leap day
    datetimeleap = datetimeleap[:-1] #drop last entry so that end date lines up with original data
    data4analysis = data4analysis.interp(time=datetimeleap) #interpolate data to new calendar with all leap days
    
    false_leap_days = []
    for i in range(len(data4analysis.time)): #find indices of false leap days
        if (data4analysis.time.values[i].month == 2) and (data4analysis.time.values[i].day == 29) and calendar.isleap(data4analysis.time.values[i].year)==False:
            false_leap_days.append(i)
    data4analysis = data4analysis.drop_isel(time=false_leap_days) #drop data for false leap days
    #print('Done with leap year conversion')
        
    return data4analysis

In [7]:
def smet_convert(data4analysis, altitude, station_name, station_id, out_name):
    #function to convert data prepared by data_prep() to .smet filetype required by SNOWPACK
    
    if os.path.exists('/glade/work/eperkins/SNOWPACK_data/Input/multiple_season/2029_2100_cubic/'+out_name+'.smet'):
        val = input('/glade/work/eperkins/SNOWPACK_data/Input/'+out_name+'.smet already exists. Overwrite current file (Yes/No):')
        if val == 'No':
            print('Skipping '+station_id+', done with execution')
        elif val == 'Yes':
            #store data in memory to access individual values
            #start = time.time()
            for var in ['TA', 'RH', 'ILWR', 'VW', 'DW', 'PSUM', 'TSG', 'TSS']:  #don't need to use .compute() on ISWR since its already a numpy array
                data4analysis[var] = data4analysis[var].compute()
            #print('Data to Memory Time: '+str({time.time() - start})) #print time it takes to store each variable

            #create pandas dataframe of variables to easily build lines
            #start = time.time()

            data_pd = pd.DataFrame({'time': pd.Series(data4analysis.time.values.astype("datetime64[ns]").astype(str)),
                               'TA': pd.Series(data4analysis.TA.values.astype(str)),
                               'RH': pd.Series(data4analysis.RH.values.astype(str)),
                               'TSG': pd.Series(data4analysis.TSG.values.astype(str)),
                               'TSS': pd.Series(data4analysis.TSS.values.astype(str)),
                               'PSUM': pd.Series(data4analysis.PSUM.values.astype(str)),
                               'VW': pd.Series(data4analysis.VW.values.astype(str)),
                               'DW': pd.Series(data4analysis.DW.values.astype(str)),
                               'ISWR': pd.Series(data4analysis.ISWR.values.astype(str)),
                               'ILWR': pd.Series(data4analysis.ILWR.values.astype(str))})
            
            data_pd['line'] = data_pd.apply(lambda row: row.time+' '+row.TA+' '+row.RH+' '+row.TSG+' '+
                                            row.TSS+' '+row.PSUM+' '+row.VW+' '+row.DW+' '+row.ISWR+' '+row.ILWR, axis = 1)
            
            #data_pd = pd.DataFrame({'time': pd.Series(data4analysis.time.values.astype("datetime64[ns]").astype(str)),
            #                   'TA': pd.Series(data4analysis.TA.values.astype(str)),
            #                   'RH': pd.Series(data4analysis.RH.values.astype(str)),
            #                   'TSG': pd.Series(data4analysis.TSG.values.astype(str)),
            #                   'TSS': pd.Series(data4analysis.TSS.values.astype(str)),
            #                   'PSUM': pd.Series(data4analysis.PSUM.values.astype(str)),
            #                   'VW': pd.Series(data4analysis.VW.values.astype(str)),
            #                   'DW': pd.Series(data4analysis.DW.values.astype(str)),
            #                   'ILWR': pd.Series(data4analysis.ILWR.values.astype(str)),
            #                   'ISWR': pd.Series(data4analysis.ISWR.values.astype(str))})

            #data_pd['line'] = data_pd.apply(lambda row: row.time+' '+row.TA+' '+row.RH+' '+row.TSG+' '+
            #                                row.TSS+' '+row.PSUM+' '+row.VW+' '+row.DW+' '+row.ILWR+' '+row.ISWR, axis = 1)

            #create the header:
            data_smet = ["" for x in range(10)]
            data_smet[0] = 'SMET 1.1 ASCII'
            data_smet[1] = '[HEADER]'
            data_smet[2] = 'station_id = '+station_id
            data_smet[3] = 'station_name = '+station_name
            data_smet[4] = 'latitude = '+str(float(data4analysis.lat))
            data_smet[5] = 'longitude = '+str(float(data4analysis.lon))
            data_smet[6] = 'altitude = '+str(altitude)
            data_smet[7] = 'nodata = -999'
            data_smet[8] = 'fields = timestamp TA RH TSG TSS PSUM VW DW ISWR ILWR'
            data_smet[9] = '[DATA]'

            #write to file
            pd.Series(data_smet).append(data_pd.line).to_csv('/glade/work/eperkins/SNOWPACK_data/Input/'+out_name+'.smet', index = False, header = False)
            
            #create initial snow profile-------------------------------------------------------------------------------       
            #make empty ascii list
            data_initial = ["" for x in range(24)]

            #create the header:
            data_initial[0] = 'SMET 1.1 ASCII'
            data_initial[1] = '[HEADER]'
            data_initial[2] = 'station_id = '+station_id
            data_initial[3] = 'station_name = '+station_name
            data_initial[4] = 'latitude = '+str(float(data4analysis.lat))
            data_initial[5] = 'longitude = '+str(float(data4analysis.lon))
            data_initial[6] = 'altitude = '+str(altitude)
            data_initial[7] = 'nodata = -999'
            data_initial[8] = 'ProfileDate = '+str(data4analysis.time[0].values.astype("datetime64[ns]"))
            data_initial[9] = 'HS_Last = 0.000'    #currently make up last snow height
            data_initial[10] = 'SlopeAngle = 0.0' #make up slope angle of zero
            data_initial[11] = 'SlopeAzi = 0.0' #make up slope azimuth of zero (pointing north)
            data_initial[12] = 'nSoilLayerData = 0' #no soil layers
            data_initial[13] = 'nSnowLayerData = 0' # no snow layers to start
            data_initial[14] = 'SoilAlbedo = 0.09' #make up soil albedo
            data_initial[15] = 'BareSoil_z0 = 0.200' #make up value for now
            data_initial[16] = 'CanopyHeight = 0.00' #no canopy
            data_initial[17] = 'CanopyLeafAreaIndex = 0.00' #no canopy
            data_initial[18] = 'CanopyDirectThroughfall = 1.00' #no canopy
            data_initial[19] = 'WindScalingFactor = 1.00' #make up for now
            data_initial[20] = 'ErosionLevel = 0' #make up for now
            data_initial[21] = 'TimeCountDeltaHS = 0.000000' #irrelevant since don't actually have any initial data
            data_initial[22] = 'fields = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo'
            data_initial[23] = '[DATA]'


            # open file in write mode
            with open(r'/glade/work/eperkins/SNOWPACK_data/Input/'+out_name+'_initial.sno', 'w') as fp_initial:
                for item in data_initial:
                    # write each item on a new line
                    fp_initial.write("%s\n" % item)
            
            #print('Time to write: '+str({time.time() - start}))
        else:
            raise ValueError('Invalid response')
    else:
        #store data in memory to access individual values
        #start = time.time()
        for var in ['TA', 'RH', 'ILWR', 'VW', 'DW', 'PSUM', 'TSG', 'TSS']:  #don't need to use .compute() on ISWR since its already a numpy array
            data4analysis[var] = data4analysis[var].compute()
        #print('Data to Memory Time: '+str({time.time() - start})) #print time it takes to store each variable

        #create pandas dataframe of variables to easily build lines
        start = time.time()

        data_pd = pd.DataFrame({'time': pd.Series(data4analysis.time.values.astype("datetime64[ns]").astype(str)),
                           'TA': pd.Series(data4analysis.TA.values.astype(str)),
                           'RH': pd.Series(data4analysis.RH.values.astype(str)),
                           'TSG': pd.Series(data4analysis.TSG.values.astype(str)),
                           'TSS': pd.Series(data4analysis.TSS.values.astype(str)),
                           'PSUM': pd.Series(data4analysis.PSUM.values.astype(str)),
                           'VW': pd.Series(data4analysis.VW.values.astype(str)),
                           'DW': pd.Series(data4analysis.DW.values.astype(str)),
                           'ISWR': pd.Series(data4analysis.ISWR.values.astype(str)),
                           'ILWR': pd.Series(data4analysis.ILWR.values.astype(str))})
            
        data_pd['line'] = data_pd.apply(lambda row: row.time+' '+row.TA+' '+row.RH+' '+row.TSG+' '+
                                        row.TSS+' '+row.PSUM+' '+row.VW+' '+row.DW+' '+row.ISWR+' '+row.ILWR, axis = 1)


        #create the header:
        data_smet = ["" for x in range(10)]
        data_smet[0] = 'SMET 1.1 ASCII'
        data_smet[1] = '[HEADER]'
        data_smet[2] = 'station_id = '+station_id
        data_smet[3] = 'station_name = '+station_name
        data_smet[4] = 'latitude = '+str(float(data4analysis.lat))
        data_smet[5] = 'longitude = '+str(float(data4analysis.lon))
        data_smet[6] = 'altitude = '+str(altitude)
        data_smet[7] = 'nodata = -999'
        data_smet[8] = 'fields = timestamp TA RH TSG TSS PSUM VW DW ISWR ILWR'
        data_smet[9] = '[DATA]'

        #write to file
        pd.Series(data_smet).append(data_pd.line).to_csv('/glade/work/eperkins/SNOWPACK_data/Input/'+out_name+'.smet', index = False, header = False)
            
        #create initial snow profile-------------------------------------------------------------------------------       
        #make empty ascii list
        data_initial = ["" for x in range(24)]

        #create the header:
        data_initial[0] = 'SMET 1.1 ASCII'
        data_initial[1] = '[HEADER]'
        data_initial[2] = 'station_id = '+station_id
        data_initial[3] = 'station_name = '+station_name
        data_initial[4] = 'latitude = '+str(float(data4analysis.lat))
        data_initial[5] = 'longitude = '+str(float(data4analysis.lon))
        data_initial[6] = 'altitude = '+str(altitude)
        data_initial[7] = 'nodata = -999'
        data_initial[8] = 'ProfileDate = '+str(data4analysis.time[0].values.astype("datetime64[ns]"))
        data_initial[9] = 'HS_Last = 0.000'    #currently make up last snow height
        data_initial[10] = 'SlopeAngle = 0.0' #make up slope angle of zero
        data_initial[11] = 'SlopeAzi = 0.0' #make up slope azimuth of zero (pointing north)
        data_initial[12] = 'nSoilLayerData = 0' #no soil layers
        data_initial[13] = 'nSnowLayerData = 0' # no snow layers to start
        data_initial[14] = 'SoilAlbedo = 0.09' #make up soil albedo
        data_initial[15] = 'BareSoil_z0 = 0.200' #make up value for now
        data_initial[16] = 'CanopyHeight = 0.00' #no canopy
        data_initial[17] = 'CanopyLeafAreaIndex = 0.00' #no canopy
        data_initial[18] = 'CanopyDirectThroughfall = 1.00' #no canopy
        data_initial[19] = 'WindScalingFactor = 1.00' #make up for now
        data_initial[20] = 'ErosionLevel = 0' #make up for now
        data_initial[21] = 'TimeCountDeltaHS = 0.000000' #irrelevant since don't actually have any initial data
        data_initial[22] = 'fields = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo'
        data_initial[23] = '[DATA]'

        # open file in write mode
        with open(r'/glade/work/eperkins/SNOWPACK_data/Input/'+out_name+'_initial.sno', 'w') as fp_initial:
            for item in data_initial:
                # write each item on a new line
                fp_initial.write("%s\n" % item)

## **Choose points and perform data preparation**

In [8]:
#Set coordiantes and altitudes for points of interest
lat_coords = [66.44,67.38,68.32,69.27,67.38,67.38,68.32,68.32]
lon_coords = [26.25,26.25,27.5,28.75,25,27.5,28.75,26.25]

#google earth altitudes
alt = [100.61, 262.57, 387, 160.07, 211.44, 170.45, 462.5, 285.1]

In [9]:
%%time

#prevent warnings from printing in output
warnings.filterwarnings("ignore")

#perform data prep and conversion for all points and all ensemble members
for run in range(1,11):
    start = time.time()
    at_data, rh_data, u_data, v_data, sw_data, lw_data, gt_data, st_data, prcp_data = read_data(run)
    run_str = ('00'+str(run))[len('00'+str(run))-3:]
    
    for pt in range(1,9):
        begin = time.time()
        data4analysis = data_prep(at_data, rh_data, u_data, v_data, sw_data, lw_data, gt_data, st_data, prcp_data, lat_coords[pt-1], lon_coords[pt-1], 0.1, '2015-08-01', '2100-06-15')
        smet_convert(data4analysis, alt[pt-1], 'CESM_Point'+str(pt), 'Point'+str(pt), 'CESM2_LE_Point'+str(pt)+'_2029_2100_LE'+run_str+'_2015')
        print('Total time for point '+str(pt)+':'+str({time.time() - begin}))
    print('\033[1mTotal time for run '+str(run)+':'+str({time.time()-start})+'\033[0m')

Total time for point 1:{783.0815351009369}
Total time for point 2:{766.782244682312}
Total time for point 3:{732.2177739143372}
Total time for point 4:{740.0409495830536}
Total time for point 5:{809.2094390392303}
Total time for point 6:{789.7881379127502}
Total time for point 7:{782.3880794048309}
Total time for point 8:{737.0128750801086}
Total time for run 2:{6270.606032848358}
CPU times: user 1h 3min 13s, sys: 1min 31s, total: 1h 4min 45s
Wall time: 1h 44min 30s
